# BBC news Topic Labeling using LDA

We are going to discuss here Latent Dirichlet Allocation and apply it on the BBC news articles dataset. The first part of article consist of a mathematical concepts that stands behind the method, explaining how it works. In the second part we will show how LDA perform in practice. Finally we will visualise some of our results.

## Introduction

Problem that we would like to talk about here is to find the best segregation of documents according to their topics. This kind of separation can be used for assigning a topic to a document (which is called topic labeling) or for general determination of compressed characteristics of a huge text data set.

Latent Dirichlet Allocation (LDA) is an unsupervised machine learning method that is a state-of-the-art approach for this kind of problems. It is based on straightforward mathematical probabilistic concept of bayesian inference, but despite its tough theory in the end it is pretty simple to use. There are various possible application of LDA: for finding questions on stack overflow that are similar to each other, news flow aggregation and analysis or any document categorization that is based on a group of structured text data. First time it was used in biology on multilocus genotype for population inference.

## 1. LDA - model description

 <h3 style="text-align: right"><i>
"No human investigation can be called real science<br> if it cannot be demonstrated mathematically. </i></h3><p style="text-align: right"> (Leonardo da Vinci) </p>

Latent Dirichlet Allocation is a generative probability model that is constructed along bayes Inference. It provides distribution of outputs and inputs based on latent variables, whereas the only observed values (grayed out in the graph below) are words. This approach requires only few assumptions to be made. One of them is the number of topics $K$ we want to distinguish. The others are $\alpha$ and $\beta$ which we will mention later.

LDA has its pythonic implementation in `sklearn` package (`sklearn.decomposition.LatentDirichletAllocation`). Number of topics should be pass through `n_components parameter`.

![alt text][logo]

<center>Plate notation of LDA model</center>

[logo]: https://upload.wikimedia.org/wikipedia/commons/4/4d/Smoothed_LDA.png

 It is easier to understand the mathematical theory that lays behind LDA model if we take a closer look on the method algorithm. It is an iterative procedure, where in each step we interpolate two kinds of multidimensional probabilistic distributions: "word for $k$-th topic distribution" $\phi_k$ and "topic for $d$-th document distribution" $\theta_d$.

In [79]:
# Example distributions

As we search for the distributions we have a general idea of how they should look like.

This _prior_ knowledge added to the model is the core of bayesian statistics and provides better performance. For distributions $\phi_k$ we expect that some words will occur often in the $k$-th topic and the ones that will be very rare,  therefore **skewed** distributions of $\phi_k$ would be appropriate. For distributions $\theta_d$ we usually assume that the topics are **uniformely** distiributed across the $d$-th document. Both situations can be reflected well using Dirichlet distribution, although with different parameters:

<center>
$\theta \sim Dir(\alpha)$, <br>
$\phi \sim Dir(\beta)$,
</center>

where:
* $\theta = (\theta_1, \theta_2, ..., \theta_d, ..., \theta_M)$,
* $\phi = (\phi_1, \phi_2, ... \phi_k, ..., \phi_K)$,
* $M$ - number of all documents,
* $K$ - number of topics

It's worth to highlight that random variables drawn from $Dir(\alpha)$ and $Dir(\beta)$ are matrices of $M$ and $K$ rows respectively. They are called \"priors\" and in `sklearn` can be provide with arguments `doc_topic_prior`, `topic_word_prior`. Of course, if we expect some topics to be more popular we can manage the $\alpha$ parameter to _bias_ the model toward those topics.

Iteration runs several times across all the documents in the corpus calculating the topics probability each time taking the output of a previous run as an input for next one. For $n$-th document we have following formula:

$$
\mathbb{P}(z_{dn} = k | z_{-(dn)}, w, \alpha, \beta) = \frac{\mathbb{P}(z_{dk}, z_{-(dk)}, w, \alpha, \beta)}{\mathbb{P}(z_{-(dk)}, w, \alpha, \beta)} = \frac{n_{dk} + \alpha_{k} }{\sum _{i}^{K}(n_{di} + \alpha_{i})} \frac{v_{kw_{dn}}+\beta_{w_{dn}}}{\sum_{i}(v_{ki}+\beta_{i})}
$$

* $n_{dk}$ is the number of times document $d$ uses topic $k$,
* $v_{kw}$ is the number of times topic $k$ uses given word $w$,
* $z_{dk}$ is a probability of $d$-th word in $k$-th document and 
* $z_{-(dk)}$ is the oposite probability.

In the above equation we assume conditional independence. This procedure is called Gibbs sampling.

It is worth pointing out that $\alpha$ and $\beta$ are in fact $M$ and $K$ dimensional vectors, $\theta$ is $K\times M$ matrix and $\phi$ is $V \times K$ matrix ($V$ stands for vocabulary).

Using the idea of generative process we assume that each document has a topic characterised by a distribution over all the words. That way we define

<center>
topic $z_{dj} \sim Multinomial(\theta_{d})$, <br>
word $w_{dj} \sim Multinomial(\phi_{z_{dj}})$
</center>

for each of the word positions $d,j$ where $d \in \{1,\dots ,M\}$, and $j \in \{1,\dots ,N_{d}\}$, where $M$ is a number of documents and $N_d$ is a length of a document $d$.
Random variable $w$ is a matrix containing probability of each document being to certain topic.

## 2. LDA using Python

Before we will start coding let us set basic notation.

<ul>

   
<li><b>Corpus</b> is a complete set of documents. In our case it will be the whole list of BBC news articles.</li>

<li><b>Document</b> is a text that has a certain topic, for us it is just a particular article.</li>

<li><b>Tokenisation</b> is the task of chopping up documents defined as character sequence into pieces, called tokens, perhaps at the same time throwing away certain characters.</li>

<li><b>Stemming</b> is a process of retriving the root of each individual word of the document.</li>

<li><b>Stopwords</b> are words that do not carry any information that would be helpful determining the document topic.</li>

</ul>

We will use following external libraries:

In [ ]:
import glob
import nltk
import sklearn
import pandas
import bokeh

### 2.1 Data

The data was downloaded from [Kaggle Datasets][1].

[1]:https://www.kaggle.com/pariza/bbc-news-summary

For corpus of our dataset we will read all the downloaded articles into one list `corpus`.
We will use the folder name as list of `labels` - this will be handy later for validation of LDA effectiveness.
If you create a python script in the same directory as the downloaded data you can use bellow code 
to import the data.

In [59]:
import glob
import os

base_dir = "BBC News Summary/News Articles"
# read news
business_file_list = glob.glob(os.path.join(os.getcwd(), base_dir, "business", "*.txt"))
entertainment_file_list = glob.glob(os.path.join(os.getcwd(), base_dir, "entertainment", "*.txt"))
politics_file_list = glob.glob(os.path.join(os.getcwd(), base_dir, "politics", "*.txt"))
sport_file_list = glob.glob(os.path.join(os.getcwd(), base_dir, "sport", "*.txt"))
tech_file_list = glob.glob(os.path.join(os.getcwd(), base_dir, "tech", "*.txt"))

labels = []
corpus = []
for file_list in [
    business_file_list, entertainment_file_list, politics_file_list, sport_file_list, tech_file_list
]:
    for file_path in file_list:
        with open(file_path, encoding="utf8", errors='ignore') as f_input:
            corpus.append((f_input.read()))
            labels.append(file_path.split('/')[-2])

### 2.2 Data preprocessing

Before we can use our data to build a model we need to prepare it to a form that will be understood by LDA algorithm. 

It can be done by building a matrix that has documents for rows and bag of words (list of all words from the corpus) for columns. That way defining a count of certain words in each document.
To achieve a better performance will also clean the input data so that the maximum amount of information can be represented as a smallest matrix possible. 

We will start with tokenization, that will split our documents into sentences and then into single words.

#### 2.2.1 Tokenisation

Since we imported the corpus, now we will tokenize each document. We will also lowercase all the words and remove non alphabetic characters from them. We do that so we can count the number of certain words in each document later on. We see now that rare words or word variations contain relevant information as they would just be the tail of our distribution.

In [60]:
import nltk
import re

regex = re.compile('[^a-zA-Z]')

def tokenize(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            clean_word = regex.sub('', word)
            tokens.append(clean_word.lower())
    return tokens

Lets see how this looks like.

In [61]:
tokenized = tokenize(corpus[500])
tokenized[:10]

['fiat',
 'mulls',
 'ferrari',
 'market',
 'listing',
 'ferrari',
 'could',
 'be',
 'listed',
 'on']

In [78]:
# Example documents word counts

#### 2.2.2 Stopwords

Now we will remove stopwords. Using the same logic we discussed while tokenizing, we now remove all the words that frequently appear in all documents. Their value for topic labeling is meaningless.

In [62]:
from nltk.corpus import stopwords as sw
stopwords = sw.words('english')
cleaned = [word for word in tokenized if word not in stopwords and word is not '']
cleaned[:10]

['fiat',
 'mulls',
 'ferrari',
 'market',
 'listing',
 'ferrari',
 'could',
 'listed',
 'stock',
 'market']

#### 2.2.3 Stemming

Next step is to stem the remaining words. Here we standardizing words to their root.

In [63]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer("english")

def stem(word):
    return stemmer.stem(word).strip()

Lets take a look how it went.

In [64]:
stemed = [stem(word) for word in cleaned]
stemed[:10]

['fiat',
 'mull',
 'ferrari',
 'market',
 'list',
 'ferrari',
 'could',
 'list',
 'stock',
 'market']

Perfect, the data is preprocessed and ready to apply `LDA` on it, but we would like to use a little improvement called `Term Frequency–Inverse Document Frequency`.

#### 2.2.4 TF-IDF

The concept of `LDA` is that we use on input a matrix of word occurrences:

$$
\omega_{t,d} = n_{t,d}
$$

While `TF-IDF` uses a product of two matrices:

$$
tf_{t,d} = \frac{n_{t,d}}{\sum_{t' \in d}n_{t',d}}
$$

<center>
        $n_{t,d}$ - occurrence number of word $t$ in document $d$ <br>
</center>

which gives the weight for each word in the document and

$$
idf(\omega) = \log \left( \frac{N}{df_t} \right)
$$

<center>
        $tf_{t}$ - number of documents containing word $t$<br>
        $N$ - number of documents
</center>

which calculates inversed weight of a word in context of all documents, promoting rare words. The result matrix

$$
\omega_{t,d} = tf_{t,d}\times\log \left(\frac{N}{df_{t}}\right)
$$

has highest values for words that are not common in the corpus.

Both `LDA` and `TF-IDF` matrices are sparse which can cause some computing problems. That fact is recognised as disadvantage, but it can not really be adressed straightforward to the method itself.

#### 2.2.5 LDA application

Let us now obtain desired matrix, basically all we need is a `TfidfVectorizer` class from `sklearn` library. We will use our previous work in parameters.

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=10000,
                                 min_df=0.05, stop_words=stopwords,
                                 use_idf=True, tokenizer=tokenize,
                                  lowercase=True, preprocessor=stem)

For more details and all parameters explanation please see the sklearn [documentation][1].

[1]:https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [67]:
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus);

#### 2.2.6 LDA

Applying LDA on prepared matrics:

In [68]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=5, random_state=42)
lda.fit(tfidf_matrix);

We will check the 10 words with highest probability for each topic topic.

In [69]:
for i,topic in enumerate(lda.components_):
    print(f'Topic #{i}:')
    print([tfidf_vectorizer.get_feature_names()[i] for i in topic.argsort()[-10:]])
    print('\n')

Topic #0:
['award', 'awards', 'party', 'brown', 'election', 'show', 'labour', 'best', 'mr', 'film']


Topic #1:
['sales', 'economy', 'bank', 'growth', 'firm', 'market', 'year', 'company', 'us', 'bn']


Topic #2:
['first', 'injury', 'players', 'match', 'cup', 'club', 'england', 'nt', 'win', 'game']


Topic #3:
['court', 'minister', 'secretary', 'scotland', 'people', 'law', 'would', 'government', 'wales', 'mr']


Topic #4:
['net', 'computer', 'phone', 'digital', 'music', 'software', 'users', 'mobile', 'technology', 'people']




Looks like the topics could be: movies or entertainment in general, economy, sport, politics and technology.

In [70]:
topic_values = lda.transform(tfidf_matrix)
doc_num, topic_num = topic_values.shape

Let us wrap it into a `Pandas` `Dataframe` so we can analyze it more efficiently.

In [71]:
import pandas as pd
df = pd.DataFrame({'document': corpus, 'label': labels, 'lda': topic_values.argmax(axis=1)})
df.groupby(['label', 'lda']).count().unstack()

document                            
lda                  0      1      2      3      4
label                                             
business           5.0  480.0    1.0   15.0    9.0
entertainment    350.0   10.0    3.0   11.0   12.0
politics         150.0    8.0    NaN  251.0    8.0
sport              6.0    4.0  472.0   28.0    1.0
tech               9.0   12.0   11.0   11.0  358.0

Comparing "lda" results to "labels" that were taken from the data set, we can see that almost all articles were correctly specified. Only politics news were divided into entertainment and politics, but still arround 40% more were assigned correctly. That uncertainty is understandable, we saw that this two topic shared the same words.

## 3. Visualisation with `bokeh`

Let's look at the matrix of probabilities for each document to better understand retrieved results.

In [72]:
prob_matrix = lda.transform(tfidf_matrix)

### 3.1 Matrix transformation

In [73]:
from math import pi
import pandas as pd

from bokeh.io import show
from bokeh.models import LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.plotting import figure
from bokeh.sampledata.unemployment1948 import data

prob_matrix_df = pd.DataFrame(data=prob_matrix[0:,0:], 
              index=list(range(len(prob_matrix))),
              columns=list(range(topic_num)))
prob_matrix_df

,0,1,2,3,4
0,0.025118,0.899510,0.025182,0.025091,0.025099
1,0.380475,0.547495,0.023812,0.024134,0.024084
2,0.025669,0.896890,0.026075,0.025574,0.025791
3,0.034627,0.834639,0.034424,0.034590,0.061720
4,0.044206,0.823558,0.043935,0.043844,0.044456
...,...,...,...,...,...
2220,0.030175,0.033565,0.044404,0.030370,0.861486
2221,0.035001,0.035894,0.392093,0.034688,0.502325
2222,0.033886,0.033625,0.033952,0.033222,0.865315
2223,0.036446,0.036475,0.036495,0.036324,0.854260


In [74]:
prob_matrix_df['doc'] = list(range(doc_num))
prob_matrix_df = prob_matrix_df.set_index('doc')
prob_matrix_df.columns.name = 'topic'
df = pd.DataFrame(prob_matrix_df.stack(), columns=['rate']).reset_index()

### 3.2 Sample data

In [75]:
import random
rand = random.sample(range(1, 2555), 100)
df_rand = df.loc[df['doc'].isin(rand)]

df_rand.to_pickle("./df_rand.pkl")
df = pd.read_pickle("./df_rand.pkl")

df['doc'] = df['doc'].astype(str)
df['topic'] = df['topic'].astype(str)
df['rate'] = df['rate']*100

docs = list(set(df_rand['doc']))
docs.sort()
docs = [str(i) for i in docs]
docs;
topics = ['0','1','2','3','4']

### 3.3 Document - topic probability plot

In [76]:
from bokeh.io import output_notebook

In [81]:
from math import pi
import pandas as pd

from bokeh.io import show, output_file
from bokeh.models import LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar
from bokeh.plotting import figure

output_file("lda_output_visualisation.html")

colors = ['#084594', '#2171b5', '#4292c6', '#6baed6', '#9ecae1', '#c6dbef', '#deebf7', '#f7fbff']
colors.reverse()

mapper = LinearColorMapper(palette=colors, low=df.rate.min(), high=df.rate.max())
TOOLS = "hover,save,pan,box_zoom,reset,wheel_zoom"

p = figure(title="Topic probability per document",
           x_range=topics, y_range=docs,
           x_axis_location="above", plot_width=400, plot_height=1200,
                      tools=TOOLS, toolbar_location='below',
           tooltips=[('rate', '@rate%')]
          )

p.grid.grid_line_color = None
p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "5pt"
p.axis.major_label_standoff = 0
p.xaxis.major_label_orientation = pi / 3

p.rect(x="topic", y="doc", width=1, height=1,
       source=df,
       fill_color={'field': 'rate', 'transform': mapper},
       line_color=None)

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="5pt",
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"),
                     label_standoff=6, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

output_notebook()

show(p)

Loading BokehJS ...

Above density plot shows, on some randomly selected documents, probability scores for each topic. It is clear now that some of the documents were almost equally classified in two categories so that the error is highly probable. That cases should be always taken into account.

## Conclusions

Big advantage of using this method among other is that it gives a score for each topic belongingness not just the topic indicator which can be helpful for determining the actual result correctness. It is quite important if we are training our model on unlabelded data.

Latent Dirichlet Allocation is pretty effective and really simple to use with `Python` which basically gives the machine learning capabilities to everyone who needs it. You can find all the code used here in our [Github][1] repository.

[1]: https://github.com/merixstudio

## Bibliography

[1] "Python Machine Learinng" - Sebastian Raschka, Vahid Mirjalili

[2] NLTK docuemntation https://www.nltk.org/

[3] Scikit learn documentation https://scikit-learn.org/

[4] Wikipedia https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation

[5] "Latent Dirichlet Allocation explained" https://www.thinkinfi.com/2019/01/lda-algorithm-steps.html